# YOLOv5 implementation

Dataset link: [trafficsign](https://www.kaggle.com/ahemateja19bec1025/trafficsignlocalizationdetectionyoloannotated)

Ref: [Do_Thuan.pdf](https://www.theseus.fi/bitstream/handle/10024/452552/Do_Thuan.pdf)

## 1. Check GPU usage status:
To use GPU: Go to Edit -> Notebook settings -> Hardware accelerator: GPU.
If we don't use GPU, we can skip this step.

In [1]:
!nvidia-smi

Wed Aug  3 03:28:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    13W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. Clone yolov5 repository:

## 2.1. Install environment for CPU (Optional):

In [ ]:
!pip3 install torch==1.9.1+cu102 torchvision==0.10.1+cu102 torchaudio===0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Operation cancelled by user


In [ ]:
%%writefile requirements.txt

# pip install -r requirements.txt

# Base ----------------------------------------
matplotlib>=3.2.2
numpy>=1.18.5
opencv-python>=4.1.2
Pillow>=7.1.2
PyYAML>=5.3.1
requests>=2.23.0
scipy>=1.4.1
# torch>=1.7.0
# torchvision>=0.8.1
tqdm>=4.41.0

# Logging -------------------------------------
tensorboard>=2.4.1
# wandb

# Plotting ------------------------------------
pandas>=1.1.4
seaborn>=0.11.0

# Export --------------------------------------
# coremltools>=4.1  # CoreML export
# onnx>=1.9.0  # ONNX export
# onnx-simplifier>=0.3.6  # ONNX simplifier
# scikit-learn==0.19.2  # CoreML quantization
# tensorflow>=2.4.1  # TFLite export
# tensorflowjs>=3.9.0  # TF.js export
# openvino-dev  # OpenVINO export

# Extras --------------------------------------
# albumentations>=1.0.3
# Cython  # for pycocotools https://github.com/cocodataset/cocoapi/issues/172
# pycocotools>=2.0  # COCO mAP
# roboflow
thop  # FLOPs computation

Writing requirements.txt


In [ ]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 596 kB 5.5 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


> Install torch with stable version to be able to run code with CPU. But the training is VERY slow 😭. So only run these commands if you can't run with GPU.

> After we use GPU a period of time, we will exceed usage limit, then we will have to wait a long time for usage limit to be reseted.

## 2.2. Clone repository:

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 10455, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 10455 (delta 6), reused 12 (delta 4), pack-reused 10437
Receiving objects: 100% (10455/10455), 10.68 MiB | 24.57 MiB/s, done.
Resolving deltas: 100% (7216/7216), done.


In [ ]:
# If we use GPU then use default requirement file
!pip3 install -r /content/yolov5/requirements.txt
%cd /content/yolov5

     |████████████████████████████████| 596 kB 5.8 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
/content/yolov5


## 3. Import neccessary libraries:

In [ ]:
import torch
import os
import shutil
import numpy as np
from IPython.display import Image, clear_output

clear_output()
print('Setup complete')

Setup complete


> If you have clone the repo twice, you can't delete folder directly. Insteads, use this code to delete the folder.

> ⚠️ Note: Use this code wisely.

In [ ]:
import shutil

# shutil.rmtree('/content/yolov5/runs/')

## 4. Mount your drive for uploaded data:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## 5. Edit data.yaml file:

In [ ]:
folder = 'traffic'
root_dir = '/content/drive/MyDrive/' + folder + '/'
num_classes = 36
data_dir = root_dir + 'Custom_data.yaml'
with open(data_dir, "w") as dataFile:
  dataFile.write(f"""
names:
- Cycle Zone
- Danger Ahead
- Deer Zone
- End of Right Road Go straight
- Give Way
- Go Left or Straight
- Go Right or Straight
- Go Straight
- Huddle Road
- Left Curve Ahead
- Left Sharp Curve
- No Entry
- No Over Taking Trucks
- No Over Taking
- No Stopping
- No Waiting
- Pedestrian
- Right Curve Ahead
- Right Sharp Curve
- Road Work
- RoundAbout
- Slippery Road
- Snow Warning Sign
- Speed Limit 100
- Speed Limit 120
- Speed Limit 20
- Speed Limit 30
- Speed Limit 50
- Speed Limit 60
- Speed Limit 70
- Speed Limit 80
- Stop
- Traffic Signals Ahead
- Truck Sign
- Turn Left
- Turn Right
nc: 36
train: {root_dir}/TrafficSignLocalizationandDetection/train/images
val: {root_dir}/TrafficSignLocalizationandDetection/valid/images
""")

In [ ]:
%cat /content/drive/MyDrive/traffic/Custom_data.yaml

## 6. Create a config file:

From yolov5s.yaml

In [ ]:
folder = 'traffic'
root_dir = '/content/drive/MyDrive/' + folder + '/'
num_classes = 36
config_dir = root_dir + 'custom_config.yaml'
with open(config_dir, "w") as dataFile:
  dataFile.write(f"""
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]
""")

In [ ]:
%cat /content/drive/MyDrive/traffic/custom_config.yaml

## 7. Train model:

> Since we don't train model with pre-trained model, the training process is very slow.

To increase the training result we can train with --img ~~1280~~ or increase batch size

In [ ]:
%%time
%cd /content/yolov5/

!python train.py --img 640 --batch 50 --epochs 100 --data /content/drive/MyDrive/traffic/Custom_data.yaml --cfg /content/drive/MyDrive/traffic/custom_config.yaml --weights '' --name traffic

/content/yolov5
train: weights=, cfg=/content/drive/MyDrive/traffic/custom_config.yaml, data=/content/drive/MyDrive/traffic/Custom_data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=50, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=traffic, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-177-g5402753 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, ancho

## 8. Inference with trained weight:


> ⚠️ Warning: Don't use a youtube link to detect. Because, Youtube recently removed the dislike features, which makes the library used by YOLOv5 broken.

For each model and dataset, adjust an appropriate confidence score. The **image size** we want to detect **SHOULD equal the size we trained the model**.

In [ ]:
%cd /content/yolov5/
!python detect.py --weight '/content/yolov5/runs/train/traffic/weights/best.pt' --save-txt --save-conf --exist-ok --img 416 --conf 0.03 --source '/content/drive/MyDrive/traffic/TrafficSignLocalizationandDetection/test'

## 9. Zip result:

Run this commend to zip the entire train folder, then we can download it easily

In [ ]:
!zip -r /content/train.zip /content/yolov5/runs/train/traffic/

In [ ]:
!python detect.py --source 'https://cdn.britannica.com/63/82563-050-3FCFC72A/Family-country-road.jpg'

detect: weights=yolov5s.pt, source=https://cdn.britannica.com/63/82563-050-3FCFC72A/Family-country-road.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
Found https://cdn.britannica.com/63/82563-050-3FCFC72A/Family-country-road.jpg locally at Family-country-road.jpg
YOLOv5 🚀 v6.0-175-gb5b56a3 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
image 1/1 /content/yolov5/Family-country-road.jpg: 448x640 4 persons, 4 bicycles, 1 sports ball, Done. (0.030s)
Speed: 0.6ms pre-process, 29.9ms inference, 17.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp3
